In [1]:
results = []

In [8]:
# gemini
import pandas as pd
import os
import sys
import json
import time
import argparse
import torch
import base64
import random
from PIL import Image
from io import BytesIO
import PIL.Image
import google.generativeai as genai
from google.api_core.exceptions import ResourceExhausted

# Classes to classify into
class_list = [
    'baby', 'bed', 'bicycle', 'chimpanzee', 'fox', 'leopard', 'man',
    'pickup_truck', 'plain', 'poppy', 'rocket', 'rose', 'snail',
    'sweet_pepper', 'table', 'telephone', 'wardrobe', 'whale', 'woman',
    'worm'
]

# Helper function to convert image to base64
def encode_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")

def classify_image_with_retry(message, model, retries=3):
    genai.configure(api_key=GENAI_KEY)
    model = genai.GenerativeModel(model)

    max_attempts = 5  # Number of retry attempts
    success = False
    attempt = 0
    while attempt < max_attempts and not success:
        try:
            response = model.generate_content(message, stream=True)
            response.resolve()
            answer = response.text.strip()
            success = True
            return answer

        except ResourceExhausted as e:
            retry_delay = 60 #+ attempt * 30
            print(f"⚠️ Quota limit reached. Waiting {retry_delay} seconds before retrying... (Attempt {attempt + 1}/{max_attempts})")
            time.sleep(retry_delay)
            attempt += 1

        except Exception as e:
            print(f"❌ Unexpected error:" + str(e))
            raise Exception(e)





root_dir = "/Users/jdieguez/Documents/master/TFM/code/cifar-fs/test/"
i = 0
MAX_IMAGES_PER_CLASS = 10

for class_name in os.listdir(root_dir):
    class_folder = os.path.join(root_dir, class_name)
    if not os.path.isdir(class_folder):
        continue

    images = os.listdir(class_folder)[:MAX_IMAGES_PER_CLASS]
    for img_name in images:
        img_path = os.path.join(class_folder, img_name)
        try:
            img = PIL.Image.open(img_path)
            text = f"This image belongs to one of the following classes:\n\n{', '.join(class_list)}.\n\nPlease classify it by returning only the most appropriate class name from the list."
            message=[text] + [img]
            
            # Initialize row for this image
            row = {
                "image_path": img_path,
                "real_class": class_name
            }
            model = 'gemini-2.0-flash-lite'
            pred = classify_image_with_retry(message, model)
            if pred == "None":
                raise Exception("It's not working")
            row[f"pred_{model}"] = pred if pred else "None"

            results.append(row)
    
            i += 1
            print(f"{i}. {row}")

        except Exception as e:
            print(f"Error with {img_path}: {e}")


1. {'image_path': '/Users/jdieguez/Documents/master/TFM/code/cifar-fs/test/man/adam_s_002297.png', 'real_class': 'man', 'pred_gemini-2.0-flash-lite': 'Man'}
⚠️ Quota limit reached. Waiting 60 seconds before retrying... (Attempt 1/5)
⚠️ Quota limit reached. Waiting 60 seconds before retrying... (Attempt 2/5)


KeyboardInterrupt: 

In [9]:
import pandas as pd

df_new = pd.DataFrame(results)

# Read existing CSV (if it exists)
try:
    df_existing = pd.read_csv("gemini_classification_results.csv")
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    print("results combined")
except FileNotFoundError:
    df_combined = df_new

df_combined.to_csv("gemini_classification_results.csv", index=False)
print("Classification completed. Results saved to gemini_classification_results.csv")

results combined
Classification completed. Results saved to gemini_classification_results.csv


In [10]:
df_combined

,image_path,real_class,pred_gemini-2.0-flash-lite
0,/Users/jdieguez/Documents/master/TFM/code/cifa...,fox,Fox
1,/Users/jdieguez/Documents/master/TFM/code/cifa...,fox,Fox
2,/Users/jdieguez/Documents/master/TFM/code/cifa...,fox,fox
3,/Users/jdieguez/Documents/master/TFM/code/cifa...,fox,fox
4,/Users/jdieguez/Documents/master/TFM/code/cifa...,fox,Fox
...,...,...,...
1021,/Users/jdieguez/Documents/master/TFM/code/cifa...,man,None
1022,/Users/jdieguez/Documents/master/TFM/code/cifa...,man,None
1023,/Users/jdieguez/Documents/master/TFM/code/cifa...,man,None
1024,/Users/jdieguez/Documents/master/TFM/code/cifa...,man,None


In [12]:
df = df_combined

df["pred"] = df["pred_gemini-2.0-flash-lite"].str.lower()
df = df[df["pred"] != "none"]
correct = (df["real_class"].str.lower() == df["pred"])
accuracy = correct.sum() / len(df)

print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9211
